# Import

In [1]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import os
import nltk
nltk.download('popular')
from spellchecker import SpellChecker
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from regex import E


torch.manual_seed(1)

import warnings
warnings.filterwarnings("ignore")

c:\Users\waqasali\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\waqasali\AppData\Roaming\nltk_data...
[nltk_data]    |   Package 

## Configuration

In [2]:
# Hyper-parameters 
input_size = 40 # number of features
hidden_size = 200 
num_classes = 3
num_epochs = 10
batch_size = 500
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset Class

In [3]:

class FeaturesDataset(Dataset):

    def __init__(self, path):
        # read data with numpy
        xy = np.loadtxt(path, delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # The first 3 columns are the class label, the rest are the features
        self.x_data = torch.from_numpy(xy[:, 3:]) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(xy[:, :3]) # size [n_samples, 3]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


# create dataset
train_dataset = FeaturesDataset('../data/processed/train_features.csv')
test_dataset = FeaturesDataset('../data/processed/test_features.csv')

# get first sample and unpack
first_data = train_dataset[0]
features, labels = first_data
print(features, labels)

# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)


# convert to an iterator and look at one random sample
train_iter = iter(train_loader)
data = train_iter.next()
features, labels = data
print(features, labels)

test_iter = iter(test_loader)
#data = test_iter.next()
#features, labels = data
#print(features, labels)


tensor([ 3.,  7., 12.,  2.,  2.,  1.,  2., 12.,  2.,  8.,  5.,  2.,  3.,  1.,
         1.,  4.,  2.,  3.,  2.,  2., 76.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]) tensor([1., 0., 0.])
tensor([[ 1.,  0.,  4.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  5.,  ...,  0.,  0.,  0.],
        [ 0.,  4.,  5.,  ...,  0.,  0.,  0.],
        ...,
        [ 2.,  0.,  5.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  3.,  ...,  0.,  0.,  0.],
        [ 2.,  2., 11.,  ...,  0.,  0.,  0.]]) tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


## Model

In [4]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size         
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## Loss & Optimizer

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() # it automatically applies the softmax to the output of last layer. 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Training

In [6]:

total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples/batch_size)
print("NUMBER OF SAMPLES = ",str(total_samples) + " NUMBER OF ITERATIONS = ",n_iterations)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        
        # here: 33297 samples, batch_size = 500, n_iters=33297/500=66.5 -> 67 iterations
        # Run the training process        
        
        #Fixing "RuntimeError: 1D target tensor expected, multi-target not supported"
        #print(outputs)
        #print(outputs.shape)
        #print(labels)
        #print(labels.shape)
        targets = np.argmax(labels, axis=1)
        #print(targets)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        if (i+1) % 50 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_iterations}], Loss: {loss.item():.4f}')

NUMBER OF SAMPLES =  33297 NUMBER OF ITERATIONS =  67
Epoch [1/10], Step [50/67], Loss: 0.9669
Epoch [2/10], Step [50/67], Loss: 0.9610
Epoch [3/10], Step [50/67], Loss: 0.8298
Epoch [4/10], Step [50/67], Loss: 0.8942
Epoch [5/10], Step [50/67], Loss: 0.8705
Epoch [6/10], Step [50/67], Loss: 0.8905
Epoch [7/10], Step [50/67], Loss: 0.8698
Epoch [8/10], Step [50/67], Loss: 0.8543
Epoch [9/10], Step [50/67], Loss: 0.8772
Epoch [10/10], Step [50/67], Loss: 0.8235


## Evaluation

In [7]:
# Test the model
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        # max returns (value, index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Model Accuracy : {acc} %')


Model Accuracy : 63.350634371395614 %


In [8]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

8803